In [3]:
import sqlite3
import pandas as pd

In [12]:
# Join louvain results with nppes, taxonomy, facility_affiliations, and TN_Hospital_Info
query = '''
SELECT l.npi, l.communityid, specialization, facility_name, `provider_organization_name_(legal_business_name)`, `provider_last_name_(legal_name)`, 
       provider_first_name, provider_middle_name
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis = pd.read_sql(query, db)
    
louvain_analysis

,npi,communityid,specialization,facility_name,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name
0,1003013160,2983,None,TENNOVA HEALTHCARE-CLARKSVILLE,None,GRABENSTEIN,WILLIAM,P.
1,1003013160,2983,None,SAINT THOMAS WEST HOSPITAL,None,GRABENSTEIN,WILLIAM,P.
2,1003013160,2983,None,None,None,GRABENSTEIN,WILLIAM,P.
3,1003013160,2983,None,None,None,GRABENSTEIN,WILLIAM,P.
4,1003013947,4727,None,TRISTAR CENTENNIAL MEDICAL CENTER,None,OTTO,ROBERT,JOEL
...,...,...,...,...,...,...,...,...
11615,1992972087,4727,None,None,INTERNAL MEDICINE ASSOCIATES OF SOUTHERN HILLS...,None,None,None
11616,1992978845,3249,None,TRISTAR STONECREST MEDICAL CENTER,None,CHILDS,ADAM,M
11617,1992985055,3300,None,None,FAMILY CARE OF MIDDLETENNESSEE,None,None,None
11618,1992985949,4727,None,None,None,BECK,ROBERT,L


In [68]:
# Display distinct npi from louvain result
query = '''
SELECT COUNT(DISTINCT npi)
FROM louvain;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    count_distinct_npi = pd.read_sql(query, db)
    
count_distinct_npi

,COUNT(DISTINCT npi)
0,8310


In [67]:
# Find top 5 most active communityid
query = '''
SELECT l.communityid, count(*) AS num_nodes, count(distinct(specialization)) AS unique_specializations_count, 
       count(distinct(l.npi)) AS providers_count
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
GROUP BY communityid
ORDER BY num_nodes DESC
LIMIT 5;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis1 = pd.read_sql(query, db)
    
louvain_analysis1

,communityid,num_nodes,unique_specializations_count,providers_count
0,4418,2480,103,1943
1,4727,2255,69,1370
2,2983,2001,71,1386
3,6328,960,62,726
4,1642,937,59,596


In [49]:
# Find top 5 most concentrated specializations within communityid 4418 (Largest Community)
query = '''
SELECT l.communityid, specialization, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 4418
GROUP BY communityid, specialization
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis2 = pd.read_sql(query, db)
    
louvain_analysis2

,communityid,specialization,num_nodes
0,4418,None,1051
1,4418,Family,171
2,4418,Cardiovascular Disease,142
3,4418,Acute Care,122
4,4418,Diagnostic Radiology,104
5,4418,Nephrology,54


In [48]:
# Find top 5 most concentrated facility_id within communityid 4418 (Largest Community)
query = '''
SELECT l.communityid, facility_name, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 4418
GROUP BY communityid, facility_name
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis2b = pd.read_sql(query, db)
    
louvain_analysis2b

,communityid,facility_name,num_nodes
0,4418,VANDERBILT UNIVERSITY MEDICAL CENTER,1076
1,4418,None,915
2,4418,WILLIAMSON MEDICAL CENTER,159
3,4418,VANDERBILT WILSON COUNTY HOSPITAL,60
4,4418,SAINT THOMAS WEST HOSPITAL,44
5,4418,VANDERBILT TULLAHOMA-HARTON HOSPITAL,35


In [30]:
# Find top 5 most concentrated specializations within communityid 4727 (Second Largest Community)
query = '''
SELECT l.communityid, specialization, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 4727
GROUP BY communityid, specialization
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis3 = pd.read_sql(query, db)
    
louvain_analysis3

,communityid,specialization,num_nodes
0,4727,None,1230
1,4727,Diagnostic Radiology,201
2,4727,Family,94
3,4727,Cardiovascular Disease,79
4,4727,Nephrology,73
5,4727,Pulmonary Disease,60


In [51]:
# Find top 5 most concentrated facility_id within communityid 4727 (Second Largest Community)
query = '''
SELECT l.communityid, facility_name, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 4727
GROUP BY communityid, facility_name
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis3b = pd.read_sql(query, db)
    
louvain_analysis3b

,communityid,facility_name,num_nodes
0,4727,None,955
1,4727,TRISTAR CENTENNIAL MEDICAL CENTER,303
2,4727,TRISTAR HENDERSONVILLE MEDICAL CENTER,141
3,4727,TRISTAR SUMMIT MEDICAL CENTER,141
4,4727,TRISTAR SKYLINE MEDICAL CENTER,135
5,4727,SAINT THOMAS WEST HOSPITAL,68


In [31]:
# Find top 5 most concentrated specializations within communityid 2983 (Third Largest Community)
query = '''
SELECT l.communityid, specialization, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 2983
GROUP BY communityid, specialization
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis4 = pd.read_sql(query, db)
    
louvain_analysis4

,communityid,specialization,num_nodes
0,2983,None,1100
1,2983,Diagnostic Radiology,157
2,2983,Family,112
3,2983,Cardiovascular Disease,66
4,2983,Pulmonary Disease,39
5,2983,Anatomic Pathology & Clinical Pathology,38


In [52]:
# Find top 5 most concentrated facility within communityid 2983 (Third Largest Community)
query = '''
SELECT l.communityid, facility_name, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 2983
GROUP BY communityid, facility_name
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis4b = pd.read_sql(query, db)
    
louvain_analysis4b

,communityid,facility_name,num_nodes
0,2983,None,967
1,2983,SAINT THOMAS WEST HOSPITAL,444
2,2983,WILLIAMSON MEDICAL CENTER,65
3,2983,SAINT THOMAS RIVER PARK HOSPITAL,61
4,2983,TRISTAR CENTENNIAL MEDICAL CENTER,55
5,2983,SAINT THOMAS HOSPITAL FOR SPINAL SURGERY,44


In [34]:
# Find top 5 most concentrated specializations within communityid 6328 (Fourth Largest Community)
query = '''
SELECT l.communityid, specialization, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 6328
GROUP BY communityid, specialization
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis5 = pd.read_sql(query, db)
    
louvain_analysis5

,communityid,specialization,num_nodes
0,6328,None,473
1,6328,Hematology & Oncology,83
2,6328,Family,71
3,6328,Diagnostic Radiology,51
4,6328,Medical Oncology,43
5,6328,Nephrology,18


In [53]:
# Find top 5 most concentrated facility_id within communityid 6328 (Fourth Largest Community)
query = '''
SELECT l.communityid, facility_name, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 6328
GROUP BY communityid, facility_name
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis5b = pd.read_sql(query, db)
    
louvain_analysis5b

,communityid,facility_name,num_nodes
0,6328,None,574
1,6328,SAINT THOMAS RUTHERFORD HOSPITAL,134
2,6328,SAINT THOMAS WEST HOSPITAL,40
3,6328,TRISTAR CENTENNIAL MEDICAL CENTER,22
4,6328,SAINT THOMAS RIVER PARK HOSPITAL,21
5,6328,TRISTAR STONECREST MEDICAL CENTER,18


In [35]:
# Find top 5 most concentrated specializations within communityid 1642 (Fifth Largest Community)
query = '''
SELECT l.communityid, specialization, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 1642
GROUP BY communityid, specialization
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis6 = pd.read_sql(query, db)
    
louvain_analysis6

,communityid,specialization,num_nodes
0,1642,None,524
1,1642,Family,90
2,1642,Diagnostic Radiology,59
3,1642,Nephrology,25
4,1642,Psychiatric/Mental Health,22
5,1642,Cardiovascular Disease,20


In [54]:
# Find top 5 most concentrated facility_id within communityid 1642 (Fifth Largest Community)
query = '''
SELECT l.communityid, facility_name, count(*) AS num_nodes
FROM louvain AS l
INNER JOIN nppes AS n1
ON l.npi = n1.npi
INNER JOIN taxonomy AS t
ON n1.primary_taxonomy = t.code
LEFT JOIN facility_affiliations AS fa
ON l.npi = fa.npi
LEFT JOIN TN_Hospital_Info AS hos
ON fa.facility_afl_ccn = hos.facility_id
WHERE communityid = 1642
GROUP BY communityid, facility_name
ORDER BY num_nodes DESC
LIMIT 6;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    louvain_analysis6b = pd.read_sql(query, db)
    
louvain_analysis6b

,communityid,facility_name,num_nodes
0,1642,None,508
1,1642,MAURY REGIONAL HOSPITAL,154
2,1642,MARSHALL MEDICAL CENTER,68
3,1642,SOUTHERN TENNESSEE REGIONAL HEALTH SYSTEM LAWR...,32
4,1642,VANDERBILT BEDFORD HOSPITAL,30
5,1642,WAYNE MEDICAL CENTER,30
